# MLAI Week 9: Classification

### Neil D. Lawrence

### 24th November 2015

In [28]:
import pods
import mlai
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


### Review

* Last time: Looked at unsupervised learning.
* Introduced latent variables, dimensionality reduction and clustering.
* This time: Classification

### Classification

* We are given a  data set containing 'inputs', $\mathbf{X}$ and 'targets', $\mathbf{y}$.
* Each data point consists of an input vector $\mathbf{x}_i$ and a class label, $y_i$.
* For binary classification assume $y_i$ should be either $1$ (yes) or $-1$ (no).
* Input vector can be thought of as features.


### Classification Examples

* Classifiying hand written digits from binary images (automatic zip code reading)
* Detecting faces in images (e.g. digital cameras).
* Who a detected face belongs to (e.g. Picasa, Facebook, DeepFace, GaussianFace)
* Classifying type of cancer given gene expression data.
* Categorization of document types (different types of news article on the internet)

### Reminder on the Term Bayesian

* We use Bayes' rule to invert probabilities in the Bayesian approach.
     * Bayesian is not named after Bayes' rule (v. common confusion). 
     * The term Bayesian refers to the treatment of the parameters as stochastic variables.
     * This approach was proposed by @Laplace:memoire74 and @Bayes:doctrine63 independently.
     * For early statisticians this was very controversial (Fisher et al).
* The use of Bayes rule does *not* imply you are being Bayesian.
    * It is just an application of the product rule of probability.


### Discrete Probability

* Algorithms based on *prediction* function and *objective* function.
* For regression the *codomain* of the functions, $f(\mathbf{X})$ was the real numbers or sometimes real vectors. 
* In classification we are given an input vector, $\mathbf{x}$, and an associated label, $y$ which either takes the value $0$ or $1$. 

Our focus has been on models where the objective function is inspired by a probabilistic analysis of the problem. In particular we've argued that we answer questions about the data set by placing probability distributions over the various quantities of interest. For the case of binary classification this will normally involve introducing probability distributions for discrete variables. Such probability distributions, are in some senses easier than those for continuous variables, in particular we can represent a probability distribution over $y$, where $y$ is binary, with one value. If we specify the probability that $y=1$ with a number that is between 0 and 1, i.e. let's say that $P(y=1) = \pi$ (here we don't mean $\pi$ the number, we are setting $\pi$ to be a variable) then we can specify the probability distribution through a table.

| y      | 0         | 1     |
|:------:|:---------:|:-----:|
| $P(y)$ | $(1-\pi)$ | $\pi$ |

Mathematically we can use a trick to implement this same table. We can use the value $y$ as a mathematical switch and write that
$$
P(y) = \pi^y (1-\pi)^{(1-y)}
$$
where our probability distribution is now written as a function of $y$. This probability distribution is known as the [Bernoulli distribution](http://en.wikipedia.org/wiki/Bernoulli_distribution). The Bernoulli distribution is a clever trick for mathematically switching between two probabilities if we were to write it as code it would be better described as
```python
def bernoulli(x, pi):
    if y_i == 1:
        return pi(x)
    else:
        return 1-pi(x)
```
If we insert $y=1$ then the function is equal to $\pi$, and if we insert $y=0$ then the function is equal to $1-\pi$. So the function recreates the table for the distribution given above.  

The probability distribution is named for [Jacob Bernoulli](http://en.wikipedia.org/wiki/Jacob_Bernoulli), the swiss mathematician. In his book Ars Conjectandi he considered the distribution and the result of a number of 'trials' under the Bernoulli distribution to form the *binomial* distribution. Below is the page where he considers Pascal's triangle in forming combinations of the Bernoulli distribution to realise the binomial distribution for the outcome of positive trials.

### Jacob Bernoulli's Bernoulli
* Bernoulli described the Bernoulli distribution in terms of an 'urn' filled with balls.
* There are red and black balls. There is a fixed number of balls in the urn.
* The portion of red balls is given by $\pi$.
* For this reason in Bernoulli's distribution there is *epistemic* uncertainty about the distribution parameter.

In [ ]:
      figure(1), clf
      plotWidth = textWidth*0.4;
      line([0 0 1 1], [1 0 0 1], 'linewidth', 3, 'color', blackColor)
      set(gca, 'xlim', [0 1], 'ylim', [0 1])
      axis equal
      axis off
      blackProb = 0.3;
      ballRadius = 0.1;
      set(gca, 'xlim', [0 1], 'ylim', [0 1])
      t = 0:pi/24:2*pi;
      rows = 4;
      cols = 1/ballRadius;
      lastRowCols = 3;
      for row = 0:rows-1
        if row == rows-1
          cols = lastRowCols;
        end
        for col = 0:cols-1
          ballX = col*2*ballRadius+ballRadius;
          ballY = row*2*ballRadius + ballRadius;
          x = ballX*ones(size(t)) + ballRadius*sin(t);
          y = ballY*ones(size(t)) + ballRadius*cos(t);
          if rand<blackProb
            ballColor = blackColor;
          else 
            ballColor = redColor;
          end
          handle = patch(x', y', ballColor);
        end
      end
      printLatexPlot('bernoulliUrn', '../../../ml/tex/diagrams/', plotWidth);
      %{
    \end{comment}
    \only<3>{\centerline{\inputdiagram{../../../ml/tex/diagrams/bernoulliUrn}}}
  \end{columns}
\end{frame}

### Thomas Bayes's Bernoulli
* Bayes described the Bernoulli distribution (he didn't call it that!) in terms of a table and two balls.
* Each ball is rolled so it comes to rest at a uniform distribution across the table.
* The first ball comes to rest at a position that is a $\pi$ times the width of table.
* After placing the first ball you consider whether a second would land to the left or the right.
* For this reason in Bayes's distribution there is considered to be *aleatoric* uncertainty about the distribution parameter.

In [ ]:
      figure(1), clf
      plotWidth = textWidth*0.4;
      
      line([0 0 1 1 0], [0 1 1 0 0], 'linewidth', 3, 'color', blackColor)
      set(gca, 'xlim', [0 1], 'ylim', [0 1])
      axis off
      printLatexPlot('bayesBilliard0', '../../../ml/tex/diagrams/', plotWidth);
      ballX = rand(1);
      ballY = 0.5;
      r = 0.1;
      t = 0:pi/24:2*pi;
      x = ballX*ones(size(t)) + r*sin(t);
      y = ballY*ones(size(t)) + r*cos(t);
      handle = patch(x', y', blackColor);
      set(gca, 'xlim', [0 1], 'ylim', [0 1])
      printLatexPlot('bayesBilliard1', '../../../ml/tex/diagrams/', plotWidth);
      line([ballX ballX], [0 1], 'linestyle', ':', 'linewidth', 3, 'color', blackColor)
      printLatexPlot('bayesBilliard2', '../../../ml/tex/diagrams/', plotWidth);
      counter = 2;
      for ballX = rand(1, 7)
        ballY = 0.5;
        counter = counter+1;
        x = ballX*ones(size(t)) + r*sin(t);
        y = ballY*ones(size(t)) + r*cos(t);
        handle = patch(x', y', redColor);
        set(gca, 'xlim', [0 1], 'ylim', [0 1])
        printLatexPlot(['bayesBilliard' num2str(counter)], '../../../ml/tex/diagrams/', plotWidth);
        delete(handle)
      end
      %{
      \end{comment}
    \column{5cm}
    \only<1>{\input{../../../ml/tex/diagrams/bayesBilliard0}}\only<2>{\input{../../../ml/tex/diagrams/bayesBilliard1}}\only<3>{\input{../../../ml/tex/diagrams/bayesBilliard2}}\only<4>{\input{../../../ml/tex/diagrams/bayesBilliard3}}\only<5>{\input{../../../ml/tex/diagrams/bayesBilliard4}}\only<6>{\input{../../../ml/tex/diagrams/bayesBilliard5}}\only<7>{\input{../../../ml/tex/diagrams/bayesBilliard6}}\only<8>{\input{../../../ml/tex/diagrams/bayesBilliard7}}\only<9>{\input{../../../ml/tex/diagrams/bayesBilliard8}}\only<10>{\input{../../../ml/tex/diagrams/bayesBilliard9}}


### Maximum Likelihood in the Bernoulli

* Assume data, $\mathbf{y}$ is binary vector length $n$. 
* Assume each value was sampled independently from the Bernoulli distribution, given probability $\pi$
$$
p(\mathbf{y}|\pi) = \prod_{i=1}^n \pi^{y_i} (1-\pi)^{1-y_i}.
$$

### Negative Log Likelihood
* Minimize the negative log likelihood
    $$E(\pi) = -\log p(\mathbf{y}|\pi) = -\sum_{i=1}^{n} y_i \log \pi - \sum_{i=1}^n (1-y_i) \log(1-\pi),$$
* Take gradient with respect to the parameter $\pi$. 
    $$\frac{\text{d}E(\pi)}{\text{d}\pi} = -\frac{\sum_{i=1}^{n} y_i}{\pi}  + \frac{\sum_{i=1}^n (1-y_i)}{1-\pi},$$

### Fixed Point

* Stationary point: set derivative to zero
    $$0 = -\frac{\sum_{i=1}^{n} y_i}{\pi}  + \frac{\sum_{i=1}^n (1-y_i)}{1-\pi},$$

* Rearrange to form
    $$(1-\pi)\sum_{i=1}^{n} y_i =   \pi\sum_{i=1}^n (1-y_i),$$

* Giving
    $$\sum_{i=1}^{n} y_i =   \pi\left(\sum_{i=1}^n (1-y_i) + \sum_{i=1}^{n} y_i\right),$$

### Solution

* Recognise that $\sum_{i=1}^n (1-y_i) + \sum_{i=1}^{n} y_i = n$ so we have
    $$\pi = \frac{\sum_{i=1}^{n} y_i}{n}$$
* Estimate the probability associated with the Bernoulli by setting it to the number of observed positives, divided by the total length of $y$. 
* Makes intiutive sense. 
* What's your best guess of probability for coin toss is heads when you get 47 heads from 100 tosses?

### Bayes' Rule Reminder

$$\text{posterior} = \frac{\text{likelihood}\times\text{prior}}{\text{marginal likelihood}}$$

* Four components:
    1. Prior distribution: represents belief about parameter values before seeing data.
    2. Likelihood: gives relation between parameters and data.
    3. Posterior distribution: represents updated belief about parameters after data is observed.
    4. Marginal likelihood: represents assessment of the quality of the model. Can be compared with other models (likelihood/prior combinations). Ratios of marginal likelihoods are known as Bayes factors.


### Naive Bayes Classifiers

* First lecture: placing probability distributions (or densities) over all the variables of interest.
* In Naive Bayes this is exactly what we do.
* Form a classification algorithm by modelling the *joint* density of our observations. 
* Need to make assumption about joint density.

### Assumptions about Density

* Make assumptions to reduce the number of parameters we need to optimise. 
* Given label data $\mathbf{y}$ and the inputs $\mathbf{X}$ could specify joint density of all potential values of $\mathbf{y}$ and $\mathbf{X}$, $p(\mathbf{y}, \mathbf{X})$. 
* If $\mathbf{X}$ and $\mathbf{y}$ are training data.
* If $\mathbf{x}^*$ is a test input and $y^*$ a test location we want
$$
p(y^*|\mathbf{X}, \mathbf{y}, \mathbf{x}^*),
$$ 

### Answer from Rules of Probability

* Compute this distribution using the product and sum rules. 
* To specify this density need the probability associated with all possible combinations of $\mathbf{y}$ and $\mathbf{X}$. There are $2^n$ possible combinations for the vector $\mathbf{y}$ and the probability for each of these combinations must be jointly specified along with the joint density of the matrix $\mathbf{X}$, as well as being able to *extend* the density for any chosen test location $\mathbf{x}^*$. 

### Naive Bayes Assumptions

* In naive Bayes we make certain simplifying assumptions that allow us to perform all of the above in practice. 

1. Data Conditional Independence
2. Feature conditional independence
3. Marginal density for $y$.

### Data Conditional Independence

* Given model parameters $\boldsymbol{\theta}$ we assume that all data points in the model are independent. 
$$
p(y^*, \mathbf{x}^*, \mathbf{y}, \mathbf{X}|\boldsymbol{\theta}) = p(y^*, \mathbf{x}^*|\boldsymbol{\theta})\prod_{i=1}^n p(y_i, \mathbf{x}_i | \boldsymbol{\theta}).
$$
* This is a conditional independence assumption.
* We made similar assumptions for regression (where $\boldsymbol{\theta} = \left\{\mathbf{w},\sigma^2\right\}$.
* Here we assume *joint* density of $\mathbf{y}$ and $\mathbf{X}$ is independent across the data given the parameters.



### Bayes Classifier

Computing posterior distribution in this case becomes easier, this is known as the 'Bayes classifier'.

### Feature Conditional Independence

* Particular to naive Bayes: assume *features* are also conditionally independent, given param *and* the label. 
    $$p(\mathbf{x}_i | y_i, \boldsymbol{\theta}) = \prod_{j=1}^p p(x_{i,j}|y_i, \boldsymbol{\theta})$$
  where $p$ is the dimensionality of our inputs.
* This is known as the *naive Bayes* assumption.
* Bayes classifier + feature conditional independence.

### Marginal Density for $y_i$

* To specify the joint distribution we also need the marginal for $p(y_i)$
    $$p(x_{i,j},y_i| \boldsymbol{\theta}) = p(x_{i,j}|y_i, \boldsymbol{\theta})p(y_i).$$
* Because $y_i$ is binary the *Bernoulli* density makes a suitable choice for our prior over $y_i$,
    $$p(y_i|\pi) = \pi^{y_i} (1-\pi)^{1-y_i}$$
  where $\pi$ now has the interpretation as being the *prior* probability that the classification should be positive. 

### Joint Density for Naive Bayes

This allows us to write down the full joint density of the training data,
$$
p(\mathbf{y}, \mathbf{X}|\boldsymbol{\theta}, \pi) = \prod_{i=1}^n \prod_{j=1}^p p(x_{i,j}|y_i, \boldsymbol{\theta})p(y_i|\pi)
$$
which can now be fit by maximum likelihood. 

### Maximum Likelihood

As normal we form our objective as the negative log likelihood,
$$
E(\boldsymbol{\theta}, \pi) = -\log p(\mathbf{y}, \mathbf{X}|\boldsymbol{\theta}, \pi) = -\sum_{i=1}^n \sum_{j=1}^p \log p(x_{i, j}|y_i, \boldsymbol{\theta}) - \sum_{i=1}^n \log p(y_i|\pi),
$$
which we note *decomposes* into two objective functions, one which is dependent on $\pi$ alone and one which is dependent on $\boldsymbol{\theta}$ alone so we have,
$$
E(\pi, \boldsymbol{\theta}) = E(\boldsymbol{\theta}) + E(\pi).
$$

### Fit Prior
Since the two objective functions are separately dependent on the parameters $\pi$ and $\boldsymbol{\theta}$ we can minimize them independently. Firstly, minimizing the Bernoulli likelihood over the labels we have, 
$$
E(\pi) = - \sum_{i=1}^n\log p(y_i|\pi) = -\sum_{i=1}^n y_i \log \pi - \sum_{i=1}^n (1-y_i) \log (1-\pi)
$$
which we already minimized above recovering 
$$
\pi = \frac{\sum_{i=1}^n y_i}{n}.
$$

### Fit Conditional 

We now need to minimize the objective associated with the conditional distributions for the features,
$$
E(\boldsymbol{\theta}) = -\sum_{i=1}^n \sum_{j=1}^p \log p(x_{i, j} |y_i, \boldsymbol{\theta}),
$$
which necessarily implies making some assumptions about the form of the conditional distributions. The right assumption will depend on the nature of our input data. For example, if we have an input which is real valued, we could use a Gaussian density and we could allow the mean and variance of the Gaussian to be different according to whether the class was positive or negative and according to which feature we were measuring. That would give us the form,
$$
p(x_{i, j} | y_i,\boldsymbol{\theta}) = \frac{1}{\sqrt{2\pi \sigma_{y_i,j}^2}} \exp \left(-\frac{(x_{i,j} - \mu_{y_i, j})^2}{\sigma_{y_i,j}^2}\right),
$$
where $\sigma_{1, j}^2$ is the variance of the density for the $j$th output and the class $y_i=1$ and $\sigma_{0, j}^2$ is the variance if the class is 0. The means can vary similarly. Our parameters, $\boldsymbol{\theta}$ would consist of all the means and all the variances for the different dimensions.

## Making Predictions

Naive Bayes has given us the class conditional densities: $p(\mathbf{x}_i | y_i, \boldsymbol{\theta})$. To make predictions with these densities we need to form the distribution given by
$$
P(y^*| \mathbf{y}, \mathbf{X}, \mathbf{x}^*, \boldsymbol{\theta})
$$

This can be computed by using the product rule. We know that
$$
P(y^*| \mathbf{y}, \mathbf{X}, \mathbf{x}^*, \boldsymbol{\theta})p(\mathbf{y}, \mathbf{X}, \mathbf{x}^*|\boldsymbol{\theta}) = p(y*, \mathbf{y}, \mathbf{X}, \mathbf{x}^*| \boldsymbol{\theta})
$$
implying that 
$$
P(y^*| \mathbf{y}, \mathbf{X}, \mathbf{x}^*, \boldsymbol{\theta}) = \frac{p(y*, \mathbf{y}, \mathbf{X}, \mathbf{x}^*| \boldsymbol{\theta})}{p(\mathbf{y}, \mathbf{X}, \mathbf{x}^*|\boldsymbol{\theta})}
$$
and we've already defined $p(y*, \mathbf{y}, \mathbf{X}, \mathbf{x}^*| \boldsymbol{\theta})$ using our conditional 

independence assumptions above 
$$
p(y^*, \mathbf{y}, \mathbf{X}, \mathbf{x}^*| \boldsymbol{\theta}) = \prod_{j=1}^p p(x^*_{j}|y^*, \boldsymbol{\theta})p(y^*|\pi)\prod_{i=1}^n \prod_{j=1}^p p(x_{i,j}|y_i, \boldsymbol{\theta})p(y_i|\pi)
$$
The other required density is $$p(\mathbf{y}, \mathbf{X}, \mathbf{x}^*|\boldsymbol{\theta})$$ which can be found from $$p(y*, \mathbf{y}, \mathbf{X}, \mathbf{x}^*| \boldsymbol{\theta})$$ using the *sum rule* of probability,
$$
p(\mathbf{y}, \mathbf{X}, \mathbf{x}^*|\boldsymbol{\theta}) = \sum_{y^*=0}^1 p(y*, \mathbf{y}, \mathbf{X}, \mathbf{x}^*| \boldsymbol{\theta}).
$$

Because of our independence assumptions that is simply equal to 
$$p(\mathbf{y}, \mathbf{X}, \mathbf{x}^*| \boldsymbol{\theta}) = \sum_{y^*=0}^1 \prod_{j=1}^p p(x^*_{j}|y^*_i, \boldsymbol{\theta})p(y^*|\pi)\prod_{i=1}^n \prod_{j=1}^p p(x_{i,j}|y_i, \boldsymbol{\theta})p(y_i|\pi).
$$
Substituting both forms in to recover our distribution over the test label conditioned on the training data we have,
$$
P(y^*| \mathbf{y}, \mathbf{X}, \mathbf{x}^*, \boldsymbol{\theta})  = \frac{\prod_{j=1}^p p(x^*_{j}|y^*_i, \boldsymbol{\theta})p(y^*|\pi)\prod_{i=1}^n \prod_{j=1}^p p(x_{i,j}|y_i, \boldsymbol{\theta})p(y_i|\pi)}{\sum_{y^*=0}^1 \prod_{j=1}^p p(x^*_{j}|y^*_i, \boldsymbol{\theta})p(y^*|\pi)\prod_{i=1}^n \prod_{j=1}^p p(x_{i,j}|y_i, \boldsymbol{\theta})p(y_i|\pi)}
$$

and we notice that all the terms associated with the training data actually cancel, the test prediction is *conditionally independent* of the training data *given* the parameters. This is a result of our conditional independence assumptions over the data points.
$$
p(y^*| \mathbf{x}^*, \boldsymbol{\theta}) = \frac{\prod_{j=1}^p p(x^*_{j}|y^*_i, \boldsymbol{\theta})p(y^*|\pi)}{\sum_{y^*=0}^1 \prod_{j=1}^p p(x^*_{j}|y^*_i, \boldsymbol{\theta})p(y^*|\pi)}
$$
This formula is also fairly straightforward to implement. First we implement the log probabilities for the Gaussian density.

### Reading

-   Chapter 5 of @Rogers:book11 up to pg 179 (Section 5.1, and 5.2 up to 5.2.2).